# Redis tutorial

## Prerequisites

### Server status

Check the status of your Redis cluster : `!sudo service redis-server status`

If the Redis cluster is inactive, `!sudo service redis-server start` to start it.

In [1]:
!sudo service redis-server status

● redis-server.service - Advanced key-value store
   Loaded: loaded (/lib/systemd/system/redis-server.service; enabled)
   Active: active (running) since Sun 2019-02-10 09:58:53 GMT; 1h 13min ago
  Process: 408 ExecStart=/usr/bin/redis-server /etc/redis/redis.conf (code=exited, status=0/SUCCESS)
 Main PID: 428 (redis-server)
   CGroup: /system.slice/redis-server.service
           └─428 /usr/bin/redis-server 127.0.0.1:6379

Feb 10 09:58:52 nosql systemd[1]: Starting Advanced key-value store...
Feb 10 09:58:53 nosql systemd[1]: Started Advanced key-value store.


### Documentation

You will find all documentation for :
* [Redis commands](https://redis.io/commands)
* [Redis python client](https://redis-py.readthedocs.io/en/latest/)

Though the tutorial asks you to use Redis commands, they all have an equivalent in the Python client library.

### Import libraries

In [2]:
import redis

In [3]:
r = redis.Redis(host='localhost', port=6379)

In [4]:
r.set('foo', 'bar')
value = r.get('foo')
print(value)

b'bar'


You can launch a terminal aside, connect to your server with a Redis client and check that the value is still present :

```bash
vagrant@nosql:~$ redis-cli
127.0.0.1:6379> get foo
"bar"
```

## I. Quick start

### Strings and numbers

**Q** : Store the string `hello world` in key `greetings` with a `SET` command. Then use a `GET` command to retrieve it.

In [5]:
r.set('greetings', 'hello world')
r.get('greetings')

b'hello world'

**Q** : Create a `connections` key with value 0. Find the `INCR` command to increment its value, then display it. 

Try to increment the value for `greetings`.

In [6]:
r.set('connections', 0)
for i in range(10):
    r.incr('connections')
r.get('connections')

b'10'

### Lists

**Q** : Create a `colors` list with strings `red`, `black` and `blue`, using the `RPUSH` command.

In [7]:
r.rpush('colors', 'red', 'black', 'blue')

3

**Q** : Display the second element of the list with `LINDEX`

In [8]:
r.lindex('colors', 2)

b'blue'

**Q** : Display the full list with `LRANGE`

In [9]:
r.lrange('colors', 0, -1)

[b'red', b'black', b'blue']

### Using sets

**Q** : Create a `nosql` set containing values `redis`, `mongodb` and `orientdb`, using the `SADD` command.

In [10]:
r.sadd('nosql', 'redis', 'mongodb', 'orientdb')

3

**Q** : Test if `mysql` is inside the set, using `SISMEMBER`

In [11]:
r.sismember('nosql', 'mysql')

False

**Q** : Add `hbase` to the set with `SADD`, then display all elements of the set with `SMEMBERS`. Try to add `redis` again and see what happens.

In [12]:
r.sadd('nosql', 'hbase')
r.sadd('nosql', 'redis')
r.smembers('nosql')

{b'hbase', b'mongodb', b'orientdb', b'redis'}

### Using sorted sets

Sorted Sets are similar to Redis Sets with the unique feature of values stored in a set. The difference is, every member of a Sorted Set is associated with a score, that is used in order to take the sorted set ordered, from the smallest to the greatest score.

**Q** : Using `ZADD`, create a sorted set `top14` with the following entries :

```
score city
10 Agen 
33 Bordeaux 
32 Brive 
29 Castres 
38 Clermont 
24 Grenoble 
26 La Rochelle 
32 Montpellier 
14 Oyonnax 
20 Pau 
40 Racing 
22 Stade Français 
36 Toulon 
36 Toulouse
```

In [13]:
r.zadd('top14', {
      'Agen':10, 
      'Bordeaux':33,
      'Brive':32,
      'Castres':29,
      'Clermont':38,
      'Grenoble':24,
      'La Rochelle':26,
      'Montpellier':32,
      'Oyonnax':14,
      'Pau':20,
      'Racing':40,
      'Stade Français':22,
      'Toulon':36,
      'Toulouse':36
})

14

**Q** : Fetch the score for `Toulon` with `ZSCORE`, and its ranking with `ZRANK`.

In [14]:
print(r.zscore('top14', 'Toulon'))
print(r.zrank('top14', 'Toulon'))

36.0
10


**Q** : `ZRANK` starts at 0 and scores are sorted from lowest to highest, so we should use the `ZREVRANK` for a true ranking of our cities.

In [15]:
print(r.zrevrank('top14', 'Toulon'))

3


**Q** : Find the commands to display :
* the 3 best teams
* teams with more than 35 points

In [16]:
print(r.zrevrange('top14', 0, 2))
print(r.zrangebyscore('top14', 35, 999))

[b'Racing', b'Clermont', b'Toulouse']
[b'Toulon', b'Toulouse', b'Clermont', b'Racing']


### Using dictionaries

**Q** : Create a dictionary `user:1` with `HMSET` with properties `id (1), name (Jean), age (22)`. Display it with `HGETALL`

In [17]:
r.hmset('user:1', {
    'id': 1,
    'name': 'Jean',
    'age': 22
})
r.hgetall('user:1')

{b'id': b'1', b'name': b'Jean', b'age': b'22'}

**Q** : Add a `city (Lyon)` property and rename the user from `Jean` to `Paul`.

## Modelling a query cache with Redis

You are modeling data from a REST query cache system with Redis.
A request is identified by the http method and its url (without the protocol).
The content of the request is stored as it is to be returned on demand.

**Q** : Insert a PUT request in the cache on http://my-api.fr/user/10 whose answer is {"id": 10, "name": "jean"}

**Q** : Insert a GET request in the cache on http://my-api.fr/user?size=2 whose answer is[{"id": 10, "name": jean}, {"id": 11, "name": "Claire"}]

**Q** : Create a set of cache request keys.

**Q** : Check if the GET request on http: //http://my-api.fr/user

**Q** : Delete the PUT request on http://my-api.fr/user/10 from the cache.

## Postquisites

The folloinwg command removes all data from your Redis cluster.

In [18]:
!redis-cli flushall

OK
